In [0]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jan  7 12:32:05 2019

@author: Krishna
"""
# Importing neccessary packages
import h5py
import tensorflow as tf
import numpy as np
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt

# Setting the path
Path='drive/My Drive/Pre-processed'
p=os.listdir(Path)

X_Dp=[]          
X_Gado=[]
X_Flair=[]      
X_T1=[]
X_T2=[]
Y_Man1=[]

X=[]
Y1=[]

# Loading the MRI Scans 
for i in p[0:14] :
    
    q=os.listdir(os.path.join(Path,i))  
    
    x=nib.load(os.path.join(Path,i,q[0]))         
    f_Dp=x.get_fdata()
    f_Dp=np.asarray(f_Dp,'float32')
    for j in range(f_Dp.shape[2]):
        slice_Dp=cv.resize(f_Dp[:,:,j],(256,256),interpolation=cv.INTER_AREA)
        if(np.sum(slice_Dp)!=0):
          #  slice_Dp=slice_Dp/(np.max(slice_Dp)+0.00001)
            slice_Dp=(slice_Dp-np.mean(slice_Dp)+0.00001)/(np.std(slice_Dp)+0.00001)
        X_Dp.append(slice_Dp)
    
      
    
    x=nib.load(os.path.join(Path,i,q[1]))
    f_Flair=x.get_fdata()
    f_Flair=np.asarray(f_Flair,'float32')
    for j in range(f_Flair.shape[2]):
        slice_Flair=cv.resize(f_Flair[:,:,j],(256,256),interpolation=cv.INTER_AREA)
        if(np.sum(slice_Flair)!=0):
           # slice_Flair=slice_Flair/(np.max(slice_Flair)+0.00001)
            slice_Flair=(slice_Flair-np.mean(slice_Flair)+0.00001)/(np.std(slice_Flair)+0.00001)
        X_Flair.append(slice_Flair)
    
    
    x=nib.load(os.path.join(Path,i,q[2]))
    f_Gado=x.get_fdata()
    f_Gado=np.asarray(f_Gado,'float32')
    for j in range(f_Gado.shape[2]):
        slice_Gado=cv.resize(f_Gado[:,:,j],(256,256),interpolation=cv.INTER_AREA)
        if(np.sum(slice_Gado)!=0):
           # slice_Gado=slice_Gado/(np.max(slice_Gado)+0.00001)
            slice_Gado=(slice_Gado-np.mean(slice_Gado)+0.00001)/(np.std(slice_Gado)+0.00001)
        X_Gado.append(slice_Gado)
    
    
    x=nib.load(os.path.join(Path,i,q[3]))
    f_Man1=x.get_fdata()
    f_Man1=np.asarray(f_Man1,'float32')
    for j in range(f_Man1.shape[2]):
        slice_Man1=cv.resize(f_Man1[:,:,j],(256,256),interpolation=cv.INTER_AREA)
        slice_Man1=np.array(slice_Man1)
        slice_Man1[slice_Man1 > 0] = 1.0
        Y_Man1.append(slice_Man1)
    
    x=nib.load(os.path.join(Path,i,q[10]))
    f_T1=x.get_fdata()
    f_T1=np.asarray(f_T1,'float32')
    for j in range(f_T1.shape[2]):
        slice_T1=cv.resize(f_T1[:,:,j],(256,256),interpolation=cv.INTER_AREA)
        if(np.sum(slice_T1)!=0):
        #    slice_T1=slice_T1/(np.max(slice_T1)+0.00001)
            slice_T1=(slice_T1-np.mean(slice_T1)+0.00001)/(np.std(slice_T1)+0.00001)
        X_T1.append(slice_T1)
    
    
    x=nib.load(os.path.join(Path,i,q[11]))
    f_T2=x.get_fdata()
    f_T2=np.asarray(f_T2,'float32')
    for j in range(f_T2.shape[2]):
        slice_T2=cv.resize(f_T2[:,:,j],(256,256),interpolation=cv.INTER_AREA)
        if(np.sum(slice_T2)!=0):
         #   slice_T2=slice_T2/(np.max(slice_T2)+0.00001)
            slice_T2=(slice_T2-np.mean(slice_T2)+0.00001)/(np.std(slice_T2)+0.00001)
        X_T2.append(slice_T2)

for i in range(len(X_Dp)):
    X_Dp[i]=X_Dp[i].T
    X_Flair[i]=X_Flair[i].T
    X_Gado[i]=X_Gado[i].T
    X_T1[i]=X_T1[i].T
    X_T2[i]=X_T2[i].T
    Y_Man1[i]=Y_Man1[i].T


for i in range(len(X_Dp)):
    slice_Dp=X_Dp[i]
    slice_Dp=slice_Dp[:,:,np.newaxis]
    
    slice_Flair=X_Flair[i]
    slice_Flair=slice_Flair[:,:,np.newaxis]
    
    slice_Gado=X_Gado[i]
    slice_Gado=slice_Gado[:,:,np.newaxis]
    
    slice_T1=X_T1[i]
    slice_T1=slice_T1[:,:,np.newaxis]
    
    slice_T2=X_T2[i]
    slice_T2=slice_T2[:,:,np.newaxis]
    
    final_slice=np.concatenate((slice_Dp,slice_Flair,slice_Gado,slice_T1,slice_T2),axis=-1)
    
    if(np.sum(final_slice)!=0):
        X.append(final_slice)
        Y1.append(Y_Man1[i])

        
        
X=np.array(X,dtype='float32')
Y1=np.array(Y1,dtype='float32')


Y1=Y1[:,:,:,np.newaxis]


#np.save("X",X)
#np.save("Y",Y1)
#
#X = np.load("G:/Multiple Scelerosis/load_data/X.npy")
#Y = np.load("G:/Multiple Scelerosis/load_data/Y.npy")

with h5py.File('preprocessed_axial_data.h5', 'w') as hf:

    hf.create_dataset('input_X', data=X[:])
    hf.create_dataset('Manual_1', data=Y1[:])

#    
with h5py.File('inputs_no_null.h5', 'w') as hf:

    hf.create_dataset('Dp', data=X_Dp[:])
    hf.create_dataset('Flair', data=X_Flair[:])
    hf.create_dataset('Gado', data=X_Gado[:])
    hf.create_dataset('T1', data=X_T1[:]) 
    hf.create_dataset('T2', data=X_T2[:])
  #  hf.create_dataset('Y_Dp', data=Y_Dp[:])
   # hf.create_dataset('Y_Flair', data=Y_Flair[:])
   # hf.create_dataset('Y_Gado', data=Y_Gado[:])
#    hf.create_dataset('Y_T1', data=Y_T1[:]) 
 #   hf.create_dataset('Y_T2', data=Y_T2[:])
    
        


    

    

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 13 18:56:04 2019

@author: Krishna Chandra
"""
import tensorflow as tf
import keras
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation 
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate


def Convolution(input_tensor,filters):
    
    x = Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same',strides=(1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 
    return x

def model(input_shape):
    
    inputs = Input((input_shape))
    
    conv_1 = Convolution(inputs,32)
    maxp_1 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_1)
    
    conv_2 = Convolution(maxp_1,64)
    maxp_2 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_2)
    
    conv_3 = Convolution(maxp_2,128)
    maxp_3 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_3)
    
    conv_4 = Convolution(maxp_3,256)
    maxp_4 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_4)
    
    conv_5 = Convolution(maxp_4,512)
    upsample_6 = UpSampling2D((2, 2)) (conv_5)
    
    conv_6 = Convolution(upsample_6,256)
    upsample_7 = UpSampling2D((2, 2)) (conv_6)
    
    upsample_7 = concatenate([upsample_7, conv_3])
    
    conv_7 = Convolution(upsample_7,128)
    upsample_8 = UpSampling2D((2, 2)) (conv_7)
    
    conv_8 = Convolution(upsample_8,64)
    upsample_9 = UpSampling2D((2, 2)) (conv_8)
    
    upsample_9 = concatenate([upsample_9, conv_1])
    
    conv_9 = Convolution(upsample_9,32)
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv_9)
    
    model = Model(inputs=[inputs], outputs=[outputs]) 
    
    return model



In [0]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jan  7 12:32:05 2019

@author: Krishna
"""
# Importing neccessary packages
import h5py
import tensorflow as tf
import numpy as np
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt

def process():
# Setting the path
    Path='drive/My Drive/Pre-processed'
    p=os.listdir(Path)

    X_Dp=[]          
    X_Gado=[]
    X_Flair=[]      
    X_T1=[]
    X_T2=[]
    Y_Man1=[]

    X=[]
    Y1=[]

# Loading the MRI Scans 
    for i in p[0:14] :
    
        q=os.listdir(os.path.join(Path,i))  
    
        x=nib.load(os.path.join(Path,i,q[0]))         
        f_Dp=x.get_fdata()
        f_Dp=np.asarray(f_Dp,'float32')
        for j in range(f_Dp.shape[2]):
            slice_Dp=cv.resize(f_Dp[:,:,j],(256,256),interpolation=cv.INTER_AREA)
            if(np.sum(slice_Dp)!=0):
          #  slice_Dp=slice_Dp/(np.max(slice_Dp)+0.00001)
                slice_Dp=(slice_Dp-np.mean(slice_Dp)+0.00001)/(np.std(slice_Dp)+0.00001)
            X_Dp.append(slice_Dp)
    
      
    
        x=nib.load(os.path.join(Path,i,q[1]))
        f_Flair=x.get_fdata()
        f_Flair=np.asarray(f_Flair,'float32')
        for j in range(f_Flair.shape[2]):
            slice_Flair=cv.resize(f_Flair[:,:,j],(256,256),interpolation=cv.INTER_AREA)
            if(np.sum(slice_Flair)!=0):
           # slice_Flair=slice_Flair/(np.max(slice_Flair)+0.00001)
                slice_Flair=(slice_Flair-np.mean(slice_Flair)+0.00001)/(np.std(slice_Flair)+0.00001)
            X_Flair.append(slice_Flair)
    
    
        x=nib.load(os.path.join(Path,i,q[2]))
        f_Gado=x.get_fdata()
        f_Gado=np.asarray(f_Gado,'float32')
        for j in range(f_Gado.shape[2]):
            slice_Gado=cv.resize(f_Gado[:,:,j],(256,256),interpolation=cv.INTER_AREA)
            if(np.sum(slice_Gado)!=0):
           # slice_Gado=slice_Gado/(np.max(slice_Gado)+0.00001)
                slice_Gado=(slice_Gado-np.mean(slice_Gado)+0.00001)/(np.std(slice_Gado)+0.00001)
            X_Gado.append(slice_Gado)
    
    
        x=nib.load(os.path.join(Path,i,q[3]))
        f_Man1=x.get_fdata()
        f_Man1=np.asarray(f_Man1,'float32')
        for j in range(f_Man1.shape[2]):
            slice_Man1=cv.resize(f_Man1[:,:,j],(256,256),interpolation=cv.INTER_AREA)
            slice_Man1=np.array(slice_Man1)
            slice_Man1[slice_Man1 > 0] = 1.0
            Y_Man1.append(slice_Man1)
    
        x=nib.load(os.path.join(Path,i,q[10]))
        f_T1=x.get_fdata()
        f_T1=np.asarray(f_T1,'float32')
        for j in range(f_T1.shape[2]):
            slice_T1=cv.resize(f_T1[:,:,j],(256,256),interpolation=cv.INTER_AREA)
            if(np.sum(slice_T1)!=0):
        #    slice_T1=slice_T1/(np.max(slice_T1)+0.00001)
                slice_T1=(slice_T1-np.mean(slice_T1)+0.00001)/(np.std(slice_T1)+0.00001)
            X_T1.append(slice_T1)
    
    
        x=nib.load(os.path.join(Path,i,q[11]))
        f_T2=x.get_fdata()
        f_T2=np.asarray(f_T2,'float32')
        for j in range(f_T2.shape[2]):
            slice_T2=cv.resize(f_T2[:,:,j],(256,256),interpolation=cv.INTER_AREA)
            if(np.sum(slice_T2)!=0):
         #   slice_T2=slice_T2/(np.max(slice_T2)+0.00001)
                slice_T2=(slice_T2-np.mean(slice_T2)+0.00001)/(np.std(slice_T2)+0.00001)
            X_T2.append(slice_T2)

    for i in range(len(X_Dp)):
        X_Dp[i]=X_Dp[i].T
        X_Flair[i]=X_Flair[i].T
        X_Gado[i]=X_Gado[i].T
        X_T1[i]=X_T1[i].T
        X_T2[i]=X_T2[i].T
        Y_Man1[i]=Y_Man1[i].T


    for i in range(len(X_Dp)):
        slice_Dp=X_Dp[i]
        slice_Dp=slice_Dp[:,:,np.newaxis]
    
        slice_Flair=X_Flair[i]
        slice_Flair=slice_Flair[:,:,np.newaxis]
    
        slice_Gado=X_Gado[i]
        slice_Gado=slice_Gado[:,:,np.newaxis]
    
        slice_T1=X_T1[i]
        slice_T1=slice_T1[:,:,np.newaxis]
    
        slice_T2=X_T2[i]
        slice_T2=slice_T2[:,:,np.newaxis]
    
        final_slice=np.concatenate((slice_Dp,slice_Flair,slice_Gado,slice_T1,slice_T2),axis=-1)
    
        if(np.sum(final_slice)!=0):
            X.append(final_slice)
            Y1.append(Y_Man1[i])

        
        
    X=np.array(X,dtype='float32')
    Y1=np.array(Y1,dtype='float32')


    Y1=Y1[:,:,:,np.newaxis]


#np.save("X",X)
#np.save("Y",Y1)
#
#X = np.load("G:/Multiple Scelerosis/load_data/X.npy")
#Y = np.load("G:/Multiple Scelerosis/load_data/Y.npy")

    with h5py.File('preprocessed_axial_data.h5', 'w') as hf:

        hf.create_dataset('input_X', data=X[:])
        hf.create_dataset('Manual_1', data=Y1[:])

#    
    with h5py.File('inputs_no_null.h5', 'w') as hf:

        hf.create_dataset('Dp', data=Dp[:])
        hf.create_dataset('Flair', data=Flair[:])
        hf.create_dataset('Gado', data=Gado[:])
        hf.create_dataset('T1', data=T1[:]) 
        hf.create_dataset('T2', data=T2[:])
     #   hf.create_dataset('Y_Dp', data=Y_Dp[:])
      #  hf.create_dataset('Y_Flair', data=Y_Flair[:])
       # hf.create_dataset('Y_Gado', data=Y_Gado[:])
       # hf.create_dataset('Y_T1', data=Y_T1[:]) 
       # hf.create_dataset('Y_T2', data=Y_T2[:])
    
        


    

    

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 13 18:52:08 2019

@author: Krishna Chandra
"""

from keras import backend as K
import numpy as np
import tensorflow as tf

# Computing Dice_Coefficient
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 13 20:18:06 2019

@author: Krishna Chandra
"""



import numpy as np
#import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers 
from sklearn.model_selection import train_test_split


#import dataPrepare as process
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'



# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual Segmentation
'''X_Dp      =   modality(Path,0)
X_Flair   =   modality(Path,1)
X_Gado    =   modality(Path,2)
X_T1      =   modality(Path,10)
X_T2      =   modality(Path,11)
Y_Manual  =   modality(Path,3)

# Removing the null samples and concatenating the 5 modalities along the 3rd dimension
X, Y = remove_null_samples(X_Dp, X_Flair, X_Gado, X_T1, X_T2, Y_Manual)
'''

# Splitting the Whole data into Training and Testing data
X_train , X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=32)

# Loding the modified U-net 
model = model(input_shape = (256,256,5))
model.summary()

checkpointer = ModelCheckpoint('Modified_UNet.h5', verbose=1)
callback_list=[checkpointer]

# Compiling the model
Adam=optimizers.adam(lr=0.001)
model.compile(optimizer=Adam, loss='binary_crossentropy', metrics=['accuracy',dice_coef,precision,sensitivity,specificity])
# Fitting the model over the data
history = model.fit(X_train,Y_train,batch_size=32,epochs=60,validation_split=0.20,verbose=1,initial_epoch=0,callbacks=callback_list)

# Saving the model
model.save('Modified_UNet.h5')
history.history

# Evaluating the model on the training and testing data 
model.evaluate(x=X_train, y=Y_train, batch_size=32 , verbose=1, sample_weight=None, steps=None)
model.evaluate(x=X_test, y=Y_test, batch_size=32, verbose=1, sample_weight=None, steps=None)

# Plotting the Graphs of Accuracy, Dice_coefficient, Loss at each epoch on Training and Testing data
#plots.Accuarcy_Graph(history)
#plots.Dice_coefficient_Graph(history)
#plots.Loss_Graph(history)













NameError: ignored